<a href="https://colab.research.google.com/github/codespaghettifier/sp500-exploration/blob/master/clean_yahoo_financials.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import json
from datetime import datetime
import random
import seaborn as sns

In [2]:
with open('data/yahoo/merged_financials.json', 'r') as json_file:
    yahoo_financials_json = json.load(json_file)

In [3]:
tickers = list(yahoo_financials_json.keys())

tickers[:10]

['A', 'AAL', 'AAP', 'AAPL', 'ABBV', 'ABC', 'ABMD', 'ABT', 'ACN', 'ADBE']

In [4]:
statement_types = list(yahoo_financials_json[tickers[0]].keys())

statement_types

['quarterly_income',
 'quarterly_balance',
 'quarterly_cashflow',
 'annual_income',
 'annual_balance',
 'annual_cashflow']

In [5]:
timetamps = [list(i.keys())[0] for i in yahoo_financials_json[tickers[0]][statement_types[0]]]
timetamps

['1690754400', '1667170800', '1675119600', '1682805600', '1635631200']

In [6]:
# Get a set of all position types present in statements by statement type
position_types_by_statement_type = {}
for type in statement_types:
    position_types_by_statement_type[type] = set()

for ticker in tickers:
    for statement_type in statement_types:
        statements = yahoo_financials_json[ticker][statement_type]
        for statement in statements:
            for position_type in list(statement.values())[0].keys():
                position_types_by_statement_type[statement_type] |= {position_type}

random.sample(position_types_by_statement_type[statement_types[0]], 10)

/tmp/ipykernel_5256/2136141112.py:13: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  random.sample(position_types_by_statement_type[statement_types[0]], 10)


['taxRateForCalcs',
 'totalExpenses',
 'netIncomeFromContinuingOperationNetMinorityInterest',
 'basicEPS',
 'pretaxIncome',
 'gainOnSaleOfSecurity',
 'interestExpenseNonOperating',
 'normalizedEBITDA',
 'basicAverageShares',
 'taxProvision']

In [7]:
# Count occurences of each position type in statemens
position_occurences = {}
for statement_type in statement_types:
    position_occurences[statement_type] = {}
    for position_type in position_types_by_statement_type[statement_type]:
        position_occurences[statement_type][position_type] = 0

for ticker in tickers:
    for statement_type in statement_types:
        statements = yahoo_financials_json[ticker][statement_type]
        for statement in statements:
            for position_type in list(statement.values())[0].keys():
                position_occurences[statement_type][position_type] += 1

random.sample(position_occurences[statement_types[0]].items(), 10)

/tmp/ipykernel_5256/408673534.py:15: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  random.sample(position_occurences[statement_types[0]].items(), 10)


[('reconciledCostOfRevenue', 2149),
 ('netIncomeFromContinuingAndDiscontinuedOperation', 2405),
 ('dilutedNIAvailtoComStockholders', 2405),
 ('earningsFromEquityInterest', 586),
 ('insuranceAndClaims', 74),
 ('depreciationIncomeStatement', 155),
 ('amortizationOfIntangiblesIncomeStatement', 517),
 ('sellingGeneralAndAdministration', 2137),
 ('securitiesAmortization', 4),
 ('otherOperatingExpenses', 770)]

In [8]:
# For each statement type get a list of position types sorted by their occurence
position_occurences_sorted = {}
for statement_type in statement_types:
    position_occurences_sorted[statement_type] = sorted(position_occurences[statement_type].items(), key=lambda x: x[1], reverse=True)

for statement_type in statement_types:
    print(statement_type)
    for position_type, occurence in position_occurences_sorted[statement_type]:
        print(f'\t{occurence}:\t {position_type}')
    print()

quarterly_income
	2411:	 basicAverageShares
	2411:	 dilutedAverageShares
	2409:	 dilutedEPS
	2409:	 basicEPS
	2405:	 netIncomeIncludingNoncontrollingInterests
	2405:	 operatingRevenue
	2405:	 pretaxIncome
	2405:	 normalizedIncome
	2405:	 netIncomeFromContinuingOperationNetMinorityInterest
	2405:	 totalRevenue
	2405:	 netIncome
	2405:	 netIncomeFromContinuingAndDiscontinuedOperation
	2405:	 netIncomeContinuousOperations
	2405:	 netIncomeCommonStockholders
	2405:	 taxEffectOfUnusualItems
	2405:	 taxRateForCalcs
	2405:	 dilutedNIAvailtoComStockholders
	2374:	 taxProvision
	2362:	 netInterestIncome
	2330:	 reconciledDepreciation
	2273:	 totalExpenses
	2269:	 ebit
	2230:	 netNonOperatingInterestIncomeExpense
	2200:	 interestExpense
	2174:	 normalizedEBITDA
	2174:	 operatingIncome
	2174:	 eBITDA
	2167:	 operatingExpense
	2149:	 grossProfit
	2149:	 costOfRevenue
	2149:	 reconciledCostOfRevenue
	2137:	 sellingGeneralAndAdministration
	2122:	 otherIncomeExpense
	2068:	 interestExpenseNonOperati

In [9]:
# For each statement type get a list of position types sorted by their occurence
position_occurences_sorted = {}
for statement_type in statement_types:
    position_occurences_sorted[statement_type] = sorted(position_occurences[statement_type].items(), key=lambda x: x[0], reverse=True)

for statement_type in statement_types:
    print(statement_type)
    for position_type, occurence in position_occurences_sorted[statement_type]:
        print(f'\t{occurence}:\t {position_type}')
    print()

quarterly_income
	244:	 writeOff
	1862:	 totalUnusualItemsExcludingGoodwill
	1862:	 totalUnusualItems
	2405:	 totalRevenue
	332:	 totalOtherFinanceCost
	1770:	 totalOperatingIncomeAsReported
	2273:	 totalExpenses
	2405:	 taxRateForCalcs
	2374:	 taxProvision
	2405:	 taxEffectOfUnusualItems
	1583:	 specialIncomeCharges
	2137:	 sellingGeneralAndAdministration
	535:	 sellingAndMarketingExpense
	4:	 securitiesAmortization
	350:	 salariesAndWages
	869:	 restructuringAndMergernAcquisition
	741:	 researchAndDevelopment
	159:	 rentExpenseSupplemental
	64:	 rentAndLandingFees
	2330:	 reconciledDepreciation
	2149:	 reconciledCostOfRevenue
	63:	 provisionForDoubtfulAccounts
	2405:	 pretaxIncome
	330:	 preferredStockDividends
	322:	 otherunderPreferredStockDividend
	215:	 otherTaxes
	513:	 otherSpecialCharges
	770:	 otherOperatingExpenses
	1825:	 otherNonOperatingIncomeExpenses
	2122:	 otherIncomeExpense
	815:	 otherGandA
	2405:	 operatingRevenue
	2174:	 operatingIncome
	2167:	 operatingExpense
	24

In [10]:
position_types_selected_for_analysis = {}
position_types_selected_for_analysis['quarterly_income'] = [
    'totalRevenue',
    'totalExpenses',
    'grossProfit',
    'netIncome',
    'ebit',
    'eBITDA',
    'operatingRevenue',
    'operatingIncome',
    'dilutedEPS',
]
position_types_selected_for_analysis['quarterly_balance'] = [
    'totalAssets',
    'totalLiabilitiesNetMinorityInterest',
    'totalDebt',
    'ordinarySharesNumber',
    'commonStockEquity',
    'tangibleBookValue',
    'investedCapital',
]
position_types_selected_for_analysis['quarterly_cashflow'] = [
    'operatingCashFlow',
    'investingCashFlow',
    'financingCashFlow',
    'freeCashFlow',
]
position_types_selected_for_analysis['annual_income'] = [
    'totalRevenue',
    'totalExpenses',
    'grossProfit',
    'netIncome',
    'ebit',
    'eBITDA',
    'operatingRevenue',
    'operatingIncome',
    'dilutedEPS',
]
position_types_selected_for_analysis['annual_balance'] = [
    'totalAssets',
    'totalLiabilitiesNetMinorityInterest',
    'totalDebt',
    'ordinarySharesNumber',
    'commonStockEquity',
    'tangibleBookValue',
    'investedCapital',
]
position_types_selected_for_analysis['annual_cashflow'] = [
    'operatingCashFlow',
    'investingCashFlow',
    'financingCashFlow',
    'freeCashFlow',
]


In [11]:
# Filter out statements missing position types selected for analysis
yahoo_financials_complete = {}
for ticker in tickers:
    yahoo_financials_complete[ticker] = {}
    for statement_type in statement_types:
        yahoo_financials_complete[ticker][statement_type] = []
        for statement in yahoo_financials_json[ticker][statement_type]:
            complete = True
            for position_type in position_types_selected_for_analysis[statement_type]:
                if position_type not in list(statement.values())[0].keys():
                    complete = False
                    break
            if complete:
                yahoo_financials_complete[ticker][statement_type].append(statement)

In [12]:
with open('data/yahoo/quotes.json', 'r') as json_file:
    quotes_and_validities = json.load(json_file)

In [13]:
# Filter out invalid quotes
valid_quotes = {}
for ticker in quotes_and_validities['validities'].keys():
    if not quotes_and_validities['validities'][ticker]:
        continue

    valid_quotes[ticker] = quotes_and_validities['quotes'][ticker]

In [14]:
# Filter out positions not selected for analysis
yahoo_financials_selected = {}
for ticker in tickers:
    yahoo_financials_selected[ticker] = {}
    for statement_type in statement_types:
        yahoo_financials_selected[ticker][statement_type] = []
        for statement in yahoo_financials_complete[ticker][statement_type]:
            timestamp = list(statement.keys())[0]
            positions = {position_type: list(statement.values())[0][position_type] for position_type in position_types_selected_for_analysis[statement_type]}
            yahoo_financials_selected[ticker][statement_type].append({
                timestamp: positions
            })

In [15]:
# Filter out statements without valid quotes
yahoo_financials_selected_with_quotes = {}
for ticker in tickers:
    if ticker not in valid_quotes.keys():
        continue

    if ticker not in yahoo_financials_selected.keys():
        continue

    yahoo_financials_selected_with_quotes[ticker] = yahoo_financials_selected[ticker]

In [16]:
# Merge financials and quotes
yahoo_financials_selected_and_quotes = yahoo_financials_selected_with_quotes.copy()
for ticker in yahoo_financials_selected_and_quotes.keys():
    yahoo_financials_selected_and_quotes[ticker]['quotes'] = valid_quotes[ticker]

In [17]:
file_path = f'data/yahoo/selected_financials_and_quotes.json'
with open(file_path, 'w') as json_file:
    json.dump(yahoo_financials_selected_and_quotes, json_file, indent=4)